Para rodar, é necessário:
- Criar pastas com os seguintes nomes: dados, img e pdfs
- Colocar os dados necessários na pasta 'dados' (caged.csv, caged_nordeste.csv, cagedest_rn_saldo_mensal.csv, porte_mensal_rn_excel.xlsx e cnaes_subclasses.csv)
    - Obs.: rodar 'build_caged.R' para obter esses dados.
- Instalar as bibliotecas que estão comentadas abaixo

In [1]:
# Parameters
setor = "Agropecu\xe1ria"


In [2]:
# !pip install weasyprint
# !pip install geobr
# !pip install adjustText

In [3]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import base64
from io import BytesIO
import weasyprint
from pathlib import Path
import matplotlib.dates as mdates
import geopandas as gpd
from shapely.geometry import box
import geobr
import matplotlib.pyplot as plt
from adjustText import adjust_text
from datetime import datetime
from pathlib import Path

In [4]:
setor = setor.encode().decode('utf-8')

# Dados

In [5]:
data = pd.read_csv("dados/caged.csv")

In [6]:
data.head()

,competênciamov,região,uf,município,seção,subclasse,saldomovimentação,cbo2002ocupação,categoria,graudeinstrução,...,unidadesaláriocódigo,valorsaláriofixo,competênciaexc,indicadordeexclusão,admitidos,desligados,NI,grande_grupamento,porte,data
0,202001,2,24,240810,G,4511102,-1,521110,101,6,...,5,103900.0,NaN,NaN,0,-1,NaN,Comércio,Micro,2020-01-01
1,202001,2,24,240810,Q,8630502,1,514320,101,7,...,5,106078.0,NaN,NaN,1,0,NaN,Serviços,Média,2020-01-01
2,202001,2,24,240810,N,8111700,-1,517410,101,3,...,5,96500.0,NaN,NaN,0,-1,NaN,Serviços,Grande,2020-01-01
3,202001,2,24,241420,I,5510801,-1,422105,999,7,...,99,NaN,NaN,NaN,0,-1,NaN,Serviços,Micro,2020-01-01
4,202001,2,24,240800,N,8299799,1,524105,101,7,...,5,103900.0,NaN,NaN,1,0,NaN,Serviços,Pequena,2020-01-01


In [7]:
data['grande_grupamento'].unique()

array(['Comércio', 'Serviços', 'Agropecuária', 'Construção', 'Indústria',
       nan], dtype=object)

In [8]:
# setor = "Serviços"

In [9]:
print(setor)

Agropecuária


In [10]:
data = data[data["grande_grupamento"]==setor]

In [11]:
subclasses = pd.read_csv("dados/cnaes_subclasses.csv",encoding='latin1', sep=';')

In [12]:
subclasses['num_cnae'] = subclasses['num_cnae'].fillna(0).astype(int)

In [13]:
subclasses.head()

,CNAE.CNAE_SUBCLASSE,num_cnae,desc_cnae
0,"7722500 - Aluguel de fitas de vídeo, DVDs e si...",7722500,"Aluguel de fitas de vídeo, DVDs e similares"
1,8630503 - Atividade médica ambulatorial restri...,8630503,Atividade médica ambulatorial restrita a consu...
2,133402 - Cultivo de banana,133402,Cultivo de banana
3,1321900 - Tecelagem de fios de algodão,1321900,Tecelagem de fios de algodão
4,8413200 - Regulação das atividades econômicas,8413200,Regulação das atividades econômicas


In [14]:
# Realizar o merge com base na correspondência entre 'num_cnae' e 'subclasse'
data = data.merge(subclasses[['num_cnae', 'desc_cnae']], 
                  how='left', 
                  left_on='subclasse', 
                  right_on='num_cnae')

In [15]:
data.head()

,competênciamov,região,uf,município,seção,subclasse,saldomovimentação,cbo2002ocupação,categoria,graudeinstrução,...,competênciaexc,indicadordeexclusão,admitidos,desligados,NI,grande_grupamento,porte,data,num_cnae,desc_cnae
0,202001,2,24,240990,A,321302,1,517420,101,1,...,NaN,NaN,1,0,NaN,Agropecuária,Micro,2020-01-01,321302.0,Criação de camarões em água salgada e salobra
1,202001,2,24,240020,A,119907,-1,622510,101,6,...,NaN,NaN,0,-1,NaN,Agropecuária,Média,2020-01-01,119907.0,Cultivo de melão
2,202001,2,24,240800,A,119908,-1,622510,101,2,...,NaN,NaN,0,-1,NaN,Agropecuária,Média,2020-01-01,119908.0,Cultivo de melancia
3,202001,2,24,240710,A,152102,-1,782510,101,7,...,NaN,NaN,0,-1,NaN,Agropecuária,Micro,2020-01-01,152102.0,Criação de equinos
4,202001,2,24,240230,A,119907,1,622505,101,2,...,NaN,NaN,1,0,NaN,Agropecuária,Média,2020-01-01,119907.0,Cultivo de melão


In [16]:
data.shape

(131725, 38)

In [17]:
data['ano'] = data['competênciamov'].astype(str).str[:4]

In [18]:
data['data'] = pd.to_datetime(data['data'])
# Encontrar a data mais nome
data_mais_recente = data['data'].max()

In [19]:
# Converter para o formato 'YYYYMM' como string
competencia = data_mais_recente.strftime('%Y%m')

In [20]:
print("Competência:", competencia)

Competência: 202505


In [21]:
import locale

# Configurar o locale para o idioma português do Brasil
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

# Converter a data mais recente para um texto por extenso
mes_referente = data_mais_recente.strftime('%B de %Y').capitalize()

# Exibir a data por extenso
print("Data por extenso:", mes_referente)


Data por extenso: Maio de 2025


In [22]:
meses_pt = {
    1: 'janeiro', 2: 'fevereiro', 3: 'março',
    4: 'abril', 5: 'maio', 6: 'junho',
    7: 'julho', 8: 'agosto', 9: 'setembro',
    10: 'outubro', 11: 'novembro', 12: 'dezembro'
}

data_mais_recente = pd.to_datetime(data['data']).max()
mes_referente = f"{meses_pt[data_mais_recente.month]} de {data_mais_recente.year}"
mes_referente = mes_referente.capitalize()


In [23]:
print("Data por extenso:", mes_referente)

Data por extenso: Maio de 2025


In [24]:
mes_nome = mes_referente.split()[0]
ano = mes_referente.split()[-1]

In [25]:
data.columns

Index(['competênciamov', 'região', 'uf', 'município', 'seção', 'subclasse',
       'saldomovimentação', 'cbo2002ocupação', 'categoria', 'graudeinstrução',
       'idade', 'horascontratuais', 'raçacor', 'sexo', 'tipoempregador',
       'tipoestabelecimento', 'tipomovimentação', 'tipodedeficiência',
       'indtrabintermitente', 'indtrabparcial', 'salário', 'tamestabjan',
       'indicadoraprendiz', 'origemdainformação', 'competênciadec',
       'indicadordeforadoprazo', 'unidadesaláriocódigo', 'valorsaláriofixo',
       'competênciaexc', 'indicadordeexclusão', 'admitidos', 'desligados',
       'NI', 'grande_grupamento', 'porte', 'data', 'num_cnae', 'desc_cnae',
       'ano'],
      dtype='object')

In [26]:
data['competênciamov'] = data['competênciamov'].astype(str)

admissoes = data.loc[(data['saldomovimentação'] == 1) & (data['competênciamov'] == competencia)]
total_admissoes = admissoes['saldomovimentação'].sum()

desligamentos = data.loc[(data['saldomovimentação'] == -1) & (data['competênciamov'] == competencia)]
total_desligamentos = -desligamentos['saldomovimentação'].sum()

saldo = data.loc[(data['competênciamov'] == competencia)]['saldomovimentação'].sum()

In [27]:
data['data'] = pd.to_datetime(data['data'])

In [28]:
data.head()

,competênciamov,região,uf,município,seção,subclasse,saldomovimentação,cbo2002ocupação,categoria,graudeinstrução,...,indicadordeexclusão,admitidos,desligados,NI,grande_grupamento,porte,data,num_cnae,desc_cnae,ano
0,202001,2,24,240990,A,321302,1,517420,101,1,...,NaN,1,0,NaN,Agropecuária,Micro,2020-01-01,321302.0,Criação de camarões em água salgada e salobra,2020
1,202001,2,24,240020,A,119907,-1,622510,101,6,...,NaN,0,-1,NaN,Agropecuária,Média,2020-01-01,119907.0,Cultivo de melão,2020
2,202001,2,24,240800,A,119908,-1,622510,101,2,...,NaN,0,-1,NaN,Agropecuária,Média,2020-01-01,119908.0,Cultivo de melancia,2020
3,202001,2,24,240710,A,152102,-1,782510,101,7,...,NaN,0,-1,NaN,Agropecuária,Micro,2020-01-01,152102.0,Criação de equinos,2020
4,202001,2,24,240230,A,119907,1,622505,101,2,...,NaN,1,0,NaN,Agropecuária,Média,2020-01-01,119907.0,Cultivo de melão,2020


In [29]:
saldo_por_data = data.groupby('data')['saldomovimentação'].sum().reset_index()
saldo_por_data


,data,saldomovimentação
0,2020-01-01,-1345
1,2020-02-01,-2323
2,2020-03-01,-1566
3,2020-04-01,-868
4,2020-05-01,-27
...,...,...
60,2025-01-01,-621
61,2025-02-01,-1049
62,2025-03-01,-2112
63,2025-04-01,-614


In [30]:
cagedest_rn_saldo_mensal =  pd.read_csv("dados/cagedest_rn_saldo_mensal.csv")
print(cagedest_rn_saldo_mensal)

           data  Saldo
0    2007-01-01    351
1    2007-02-01  -1394
2    2007-03-01  -1076
3    2007-04-01   1615
4    2007-05-01   1355
..          ...    ...
151  2019-08-01   3951
152  2019-09-01   2513
153  2019-10-01   3025
154  2019-11-01   1597
155  2019-12-01  -3133

[156 rows x 2 columns]


In [31]:
porte_mensal_rn_excel =  pd.read_excel("dados/porte_mensal_rn_excel.xlsx")
print(porte_mensal_rn_excel)

          data    porte  Saldo
0   2020-01-01   Grande  -1692
1   2020-01-01    Micro   1685
2   2020-01-01    Média   -870
3   2020-01-01  Pequena   -197
4   2020-02-01   Grande  -3293
..         ...      ...    ...
255 2025-04-01  Pequena    -58
256 2025-05-01   Grande    -92
257 2025-05-01    Micro   1814
258 2025-05-01    Média    232
259 2025-05-01  Pequena    266

[260 rows x 3 columns]


In [32]:
caged_nordeste =  pd.read_csv("dados/caged_nordeste_setor.csv")
print(caged_nordeste)

     uf seção  Saldo    Estado grande_grupamento
0    21     A    -86  Maranhão      Agropecuária
1    21     B     25  Maranhão         Indústria
2    21     C    534  Maranhão         Indústria
3    21     D    -11  Maranhão         Indústria
4    21     E     29  Maranhão         Indústria
..   ..   ...    ...       ...               ...
175  29     P    774     Bahia          Serviços
176  29     Q    921     Bahia          Serviços
177  29     R    206     Bahia          Serviços
178  29     S    322     Bahia          Serviços
179  29     T      1     Bahia          Serviços

[180 rows x 5 columns]


In [33]:
# Estoques fixos
estoque_agro = 19187
estoque_industria = 80573
estoque_construcao = 37483
estoque_comercio = 129072
estoque_servicos = 235606

# Atribuir estoque com base no setor escolhido
estoque_map = {
    "Agropecuária": estoque_agro,
    "Indústria": estoque_industria,
    "Construção": estoque_construcao,
    "Comércio": estoque_comercio,
    "Serviços": estoque_servicos
}

# Define o valor de referência com base no setor
estoque_referencia_dez_2023 = estoque_map.get(setor, 0)


In [34]:
# # Estoque de referência em dezembro de 2023
# estoque_referencia_dez_2023 = 235606

# Filtrar e calcular o saldo a partir de 2024-01-01
saldo_total = saldo_por_data[saldo_por_data['data'] >= "2024-01-01"]['saldomovimentação'].sum()

# Calcular o estoque final
estoque = estoque_referencia_dez_2023 + saldo_total

# Exibir o resultado
print("Estoque:", estoque)

Estoque: 16191


In [35]:
# Ajustando formato dos números
total_admissoes = f"{total_admissoes:,}".replace(",", ".")
saldo = f"{saldo:,}".replace(",", ".")
total_desligamentos = f"{total_desligamentos:,}".replace(",", ".")
estoque = f"{estoque:,}".replace(",", ".")

In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calendar

def criar_grafico_acumulado_ano_e_mes_mais_recente(data):
    # Garantir que a coluna 'data' esteja no formato datetime
    data['data'] = pd.to_datetime(data['data'], errors='coerce')

    # Criar uma nova coluna 'ano' e 'mes' para agrupamento
    data['ano'] = data['data'].dt.year
    data['mes'] = data['data'].dt.month

    # Identificar o mês mais recente da base
    mes_mais_recente = data['data'].max().month
    nome_mes_mais_recente = calendar.month_name[mes_mais_recente]  # Nome do mês (ex: Novembro)

    # Filtrar os últimos 4 anos
    ano_atual = data['ano'].max()
    anos_filtrados = list(range(ano_atual - 4, ano_atual + 1))  # Últimos 4 anos
    data_filtrada = data[data['ano'].isin(anos_filtrados)]

    # Calcular o saldo acumulado do ano até o mês mais recente
    saldo_acumulado = data_filtrada[data_filtrada['mes'] <= mes_mais_recente].groupby('ano')['saldomovimentação'].sum().reset_index()
    saldo_acumulado.rename(columns={'saldomovimentação': 'Saldo acumulado'}, inplace=True)


    # Calcular o saldo do mês mais recente
    saldo_mes = data_filtrada[data_filtrada['mes'] == mes_mais_recente].groupby('ano')['saldomovimentação'].sum().reset_index()
    saldo_mes.rename(columns={'saldomovimentação': f'Saldo {nome_mes_mais_recente}'}, inplace=True)

    # Unir os dois conjuntos de dados
    merged_data = pd.merge(saldo_acumulado, saldo_mes, on='ano', how='outer').fillna(0)

    # Transformar os dados para formato longo (long format) para o gráfico
    melted_data = pd.melt(merged_data, id_vars='ano', value_vars=['Saldo acumulado', f'Saldo {nome_mes_mais_recente}'],
                          var_name='Tipo', value_name='Saldo')

    # Criar o gráfico de barras
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x='ano', y='Saldo', hue='Tipo', data=melted_data, palette='Blues_d')

    # Remover o título da legenda
    ax.legend(title=None)

    # Títulos e rótulos
    plt.xlabel("Ano", fontsize=12)
    plt.ylabel("Saldo", fontsize=12)

    # Adicionar valores no topo de cada barra com a cor da barra
    for p in ax.patches:
        height = p.get_height()
        color = p.get_facecolor()  # Obtém a cor da barra
        if height != 0:  # Evitar exibir "0"
            ax.text(
                p.get_x() + p.get_width() / 2.,  # Posição horizontal
                height,  # Posição vertical (no topo da barra)
                '{:,.0f}'.format(height).replace(',', '.'),  # Formatação do número
                ha='center', va='bottom' if height >= 0 else 'top', fontsize=10, fontweight='bold',
                color=color  # Define a cor do texto igual à cor da barra
            )

    # Adicionar informações de fonte e elaboração
    plt.figtext(0.05, -0.01, 'Fonte: Novo Caged (MTE).',
                ha='left', fontsize=8, color='gray', weight='normal')
    plt.figtext(0.05, -0.03, 'Elaboração: SEBRAE/RN.',
                ha='left', fontsize=8, color='gray', weight='normal')

    # Ajustar layout
    plt.tight_layout()

    # Salvar o gráfico
    grafico_path = f'img/grafico_acumulado_ano_e_mes_{setor}.png'
    plt.savefig(grafico_path, format='png', bbox_inches='tight', transparent=True)
    plt.show()

    return grafico_path

# Exemplo de chamada da função
# Suponha que `data` seja um DataFrame previamente carregado
grafico_path = criar_grafico_acumulado_ano_e_mes_mais_recente(data)
print(f"Gráfico salvo em: {grafico_path}")


Gráfico salvo em: img/grafico_acumulado_ano_e_mes_Agropecuária.png


C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_27008\1163949516.py:75: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [37]:
data['data'] = pd.to_datetime(data['data'], errors='coerce')

data['ano'] = data['data'].dt.year
data['mes'] = data['data'].dt.month

    # Identificar o mês mais recente da base
mes_mais_recente = data['data'].max().month
nome_mes_mais_recente = calendar.month_name[mes_mais_recente]  # Nome do mês (ex: Novembro)

    # Filtrar os últimos 4 anos
ano_atual = data['ano'].max()
anos_filtrados = list(range(ano_atual - 4, ano_atual + 1))  # Últimos 4 anos
data_filtrada = data[data['ano'].isin(anos_filtrados)]

    # Calcular o saldo acumulado do ano
saldo_acumulado = data_filtrada.groupby('ano')['saldomovimentação'].sum().reset_index()
saldo_acumulado.rename(columns={'saldomovimentação': 'Saldo acumulado'}, inplace=True)

In [38]:
# Agrupar os dados por ano e mês
saldo_por_mes = data_filtrada.groupby(['ano', 'mes'])['saldomovimentação'].sum().reset_index()

# Adicionar o nome do mês para facilitar a leitura
saldo_por_mes['nome_mes'] = saldo_por_mes['mes'].apply(lambda x: calendar.month_name[x])

# Ordenar os dados por ano e mês
saldo_por_mes = saldo_por_mes.sort_values(by=['ano', 'mes']).reset_index(drop=True)

# Exibir o resultado
print(saldo_por_mes)


     ano  mes  saldomovimentação   nome_mes
0   2021    1               -997    janeiro
1   2021    2              -1918  fevereiro
2   2021    3              -1574     marÃ§o
3   2021    4               -235      abril
4   2021    5                165       maio
5   2021    6               1167      junho
6   2021    7               1186      julho
7   2021    8               2420     agosto
8   2021    9               1635   setembro
9   2021   10                384    outubro
10  2021   11               -400   novembro
11  2021   12               -927   dezembro
12  2022    1              -1478    janeiro
13  2022    2              -1277  fevereiro
14  2022    3              -2105     marÃ§o
15  2022    4               -602      abril
16  2022    5                 -4       maio
17  2022    6                746      junho
18  2022    7                837      julho
19  2022    8               2927     agosto
20  2022    9               1499   setembro
21  2022   10                241

In [39]:
soma_2024 = saldo_por_mes[saldo_por_mes['ano'] == 2024]['saldomovimentação'].sum()

# Exibir o resultado
print(f"Soma total do saldo de 2024: {soma_2024}")

Soma total do saldo de 2024: 1044


In [40]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import pandas as pd

def criar_grafico_ts(data):
    # Certifique-se de que a coluna 'data' é do tipo datetime
    if not pd.api.types.is_datetime64_any_dtype(data['data']):
        data['data'] = pd.to_datetime(data['data'])

    # Agrupamento dos dados
    grouped_data = data.groupby('data')['saldomovimentação'].sum().reset_index()

    plt.figure(figsize=(12, 6))
    sns.lineplot(x='data', y='saldomovimentação', data=grouped_data)
    plt.title("")
    plt.xlabel("")
    plt.ylabel("")

    # Configurações do eixo X para datas
    ax = plt.gca()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # Exibe de 3 em 3 meses
    plt.xticks(rotation=45, fontsize=8)  # Diminui o tamanho da fonte para 8

    # Ajuste de espaçamento do gráfico
    plt.tight_layout()

    # Adicionar valores nos pontos
    last_y = None
    for x, y in zip(grouped_data['data'], grouped_data['saldomovimentação']):
        color = '#134b5f' if y >= 0 else '#be4d25'
        offset = 100 if y >= 0 else -200

        # Ajuste dinâmico para evitar sobreposição
        if last_y is not None and abs(y - last_y) < 500:
            offset = -200 if y > last_y else 300

        plt.text(x, y + offset, '{:,.0f}'.format(y).replace(',', '.'),
                 ha='center', va='bottom', fontsize=8, color=color)
        last_y = y + offset

    # Notas no rodapé
    plt.figtext(0.78, -0.01, 'Fonte: Novo Caged (MTE).',
                ha='left', fontsize=8, color='gray', weight='normal')
    plt.figtext(0.78, -0.03, 'Elaboração: SEBRAE/RN.',
                ha='left', fontsize=8, color='gray', weight='normal')

    # Salvar gráfico
    grafico_path = f'img/grafico_saldo_ts_{setor}.png'
    plt.savefig(grafico_path, format='png', bbox_inches='tight', transparent=True)
    plt.show()

    return grafico_path

# Uso da função
grafico_ts = criar_grafico_ts(data)


C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_27008\3374764869.py:52: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [41]:
# Filtra pela competência desejada
filtro = data[data['ano'] == int(ano)]

filtro['desc_cnae'] = filtro['desc_cnae'].str.split(',').str[0].str.strip()

# Agrupa apenas por CNAE, somando os saldos
grupo_saldos = filtro.groupby('desc_cnae')['saldomovimentação'].sum().reset_index()

# Ordena por saldo decrescente e pega os primeiros (por exemplo, top 10)
top_cnaes_df = grupo_saldos.sort_values('saldomovimentação', ascending=False).head(5)

# Renomeia colunas
top_cnaes_df = top_cnaes_df.rename(columns={
    'desc_cnae': 'CNAE',
    'saldomovimentação': 'Saldo de Movimentação'
})

top_cnaes_df


C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_27008\1098292130.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtro['desc_cnae'] = filtro['desc_cnae'].str.split(',').str[0].str.strip()


,CNAE,Saldo de Movimentação
55,Produção de ovos,59
15,Criação de camarões em água salgada e salobra,56
37,Cultivo de manga,50
6,Atividades de pós-colheita,42
26,Cultivo de banana,30


In [42]:
# Filtra pela competência desejada
filtro = data[data['ano'] == int(ano)]

# Pega só o que vem antes da vírgula no nome do CNAE
filtro['desc_cnae'] = filtro['desc_cnae'].str.split(',').str[0].str.strip()

# Agrupa por CNAE, somando os saldos
grupo_saldos = filtro.groupby('desc_cnae')['saldomovimentação'].sum().reset_index()

# Ordena por saldo crescente e pega o CNAE com menor saldo
bottom_cnaes_df = grupo_saldos.sort_values('saldomovimentação', ascending=True).head(5)

# Renomeia as colunas para manter o padrão
bottom_cnaes_df = bottom_cnaes_df.rename(columns={
    'desc_cnae': 'CNAE',
    'saldomovimentação': 'Saldo de Movimentação'
})

bottom_cnaes_df

C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_27008\3549525597.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtro['desc_cnae'] = filtro['desc_cnae'].str.split(',').str[0].str.strip()


,CNAE,Saldo de Movimentação
39,Cultivo de melão,-3125
28,Cultivo de cana-de-açúcar,-466
60,Serviço de preparação de terreno,-300
45,Cultivo de outras plantas de lavoura temporári...,-206
34,Cultivo de frutas de lavoura permanente não es...,-133


In [43]:
import matplotlib.pyplot as plt

def grafico_top_bottom_cnaes(top_cnaes_df, bottom_cnaes_df, titulo='Top e Bottom CNAEs por Saldo'):
    def quebra_linha(texto, max_chars=40):
        if len(texto) > max_chars:
            partes = []
            while len(texto) > max_chars:
                corte = texto[:max_chars].rfind(' ')
                if corte == -1:
                    corte = max_chars
                partes.append(texto[:corte])
                texto = texto[corte:].lstrip()
            partes.append(texto)
            return '\n'.join(partes)
        else:
            return texto

    # Cópias para evitar warning
    top_cnaes_df = top_cnaes_df.copy()
    bottom_cnaes_df = bottom_cnaes_df.copy()
    top_cnaes_df['Tipo'] = 'Top'
    bottom_cnaes_df['Tipo'] = 'Bottom'
    df_plot = pd.concat([top_cnaes_df, bottom_cnaes_df], ignore_index=True)
    df_plot = df_plot.sort_values('Saldo de Movimentação', ascending=True)
    df_plot['CNAE'] = df_plot['CNAE'].apply(lambda x: quebra_linha(x, 40))

    cores = df_plot['Tipo'].map({'Top': '#154c79', 'Bottom': '#bd0909'})
    
    plt.figure(figsize=(12, max(5, len(df_plot)*0.55)))
    bars = plt.barh(df_plot['CNAE'], df_plot['Saldo de Movimentação'], color=cores)
    plt.yticks(fontsize=8)
    
# Texto dentro ou fora da barra, com cor da própria barra
    for i, bar in enumerate(bars):
        valor = int(df_plot.iloc[i]['Saldo de Movimentação'])
        largura = bar.get_width()
        
        # Verifica se valor está na faixa entre -120 e 120
        if -120 <= valor <= 120:
            deslocamento = 4 if largura >= 0 else -4
            ha = 'left' if largura >= 0 else 'right'
            cor_texto = cores.iloc[i]  # usa a cor da própria barra
        else:
            deslocamento = -0.05 * largura
            ha = 'right' if largura > 0 else 'left'
            cor_texto = 'white' if abs(largura) >= 40 else cores.iloc[i]
            if abs(largura) < 40:
                deslocamento = 2 if largura < 0 else -2

        plt.text(
            largura + deslocamento,
            bar.get_y() + bar.get_height() / 2,
            f'{valor:,.0f}'.replace(',', '.'),
            va='center', ha=ha, fontsize=10,
            fontweight='bold', color=cor_texto
        )
    # Salvar o gráfico
    grafico_path = f'img/grafico_top_bottom_cnaes_{setor}.png'
    plt.savefig(grafico_path, format='png', bbox_inches='tight', transparent=True)

    plt.xlabel('Saldo de Movimentação')
    plt.ylabel('CNAE')
    plt.title(titulo)
    plt.axvline(0, color='black', linewidth=1)
    plt.tight_layout()
    plt.figtext(0.01, -0.03, 'Fonte: Novo Caged (MTE).', ha='left', fontsize=8, color='gray')
    plt.figtext(0.01, -0.05, 'Elaboração: SEBRAE/RN.', ha='left', fontsize=8, color='gray')
    plt.show()

    return grafico_path

grafico_top_bottom_cnaes(top_cnaes_df, bottom_cnaes_df, titulo='Top 5 Maiores e Menores Atividades Econômicas por Saldo')


C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_27008\3338683462.py:68: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


'img/grafico_top_bottom_cnaes_Agropecuária.png'

In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import os

def criar_grafico_mes_anos_anteriores(data, setor):
    data['data'] = pd.to_datetime(data['data'], errors='coerce')

    mes_mais_recente = data['data'].max().month
    nome_mes_mais_recente = calendar.month_name[mes_mais_recente]

    data_filtrada = data[(data['data'].dt.month == mes_mais_recente) & (data['data'].dt.year >= 2020)]

    data_grouped = data_filtrada.groupby(data_filtrada['data'].dt.year)['saldomovimentação'].sum().reset_index()
    data_grouped.rename(columns={'saldomovimentação': 'Saldo', 'data': 'Ano'}, inplace=True)
    data_grouped.rename(columns={data_grouped.columns[0]: 'Ano'}, inplace=True)

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x='Ano', y='Saldo', data=data_grouped, color="#1574bc")

    plt.xlabel(f"Ano ({nome_mes_mais_recente})", fontsize=12)
    plt.ylabel("Saldo de Movimentações", fontsize=12)

    for p in ax.patches:
        height = p.get_height()
        cor = '#1574bc' if height >= 0 else '#d62728'
        va = 'bottom' if height >= 0 else 'top'
        ax.text(
            p.get_x() + p.get_width() / 2.,
            height,
            '{:,.0f}'.format(height).replace(',', '.'),
            ha='center', va=va, fontsize=10, fontweight='bold', color=cor
        )

    plt.figtext(0.1, -0.01, 'Fonte: Novo Caged (MTE).', ha='left', fontsize=8, color='gray')
    plt.figtext(0.1, -0.03, 'Elaboração: SEBRAE/RN.', ha='left', fontsize=8, color='gray')

    os.makedirs("img", exist_ok=True)
    grafico_path = f'img/grafico_mes_colunas_{setor}.png'
    plt.savefig(grafico_path, format='png', bbox_inches='tight', transparent=True)
    plt.close()  # evitar múltiplos gráficos em notebook/pipeline

    return grafico_path

grafico_path = criar_grafico_mes_anos_anteriores(data, setor)
print(f"Gráfico salvo em: {grafico_path}")

Gráfico salvo em: img/grafico_mes_colunas_Agropecuária.png


In [45]:
def criar_grafico_saldo_porte(data):
    data['competênciamov'] = data['competênciamov'].astype(str)
    # Filtrar os dados onde o ano da 'competênciamov' é igual ao ano de 'competencia'
    ano_competencia = str(competencia)[:4]
    data_ano = data[data['competênciamov'].str[:4] == ano_competencia]
    # Agrupar os dados por 'porte' e somar o saldo de movimentação
    grouped_data = data_ano.groupby('porte')['saldomovimentação'].sum().reset_index()

    # Ordenar os dados para exibir os maiores valores primeiro, se necessário
    grouped_data = grouped_data.sort_values(by='saldomovimentação', ascending=False)

    # Criar o gráfico de barras
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x='porte', y='saldomovimentação', data=grouped_data, palette='Blues_d')

    # Títulos e rótulos
    # plt.title("Saldo de Movimentação por Setor")
    plt.ylabel("")
    plt.xlabel('')  # Remove o nome do eixo X

    # Adicionar a linha horizontal no valor 0 do eixo Y
    plt.axhline(0, color='black',linewidth=1)

    # Adicionar os valores de cada barra dentro delas com formatação
    for p in ax.patches:
        # Posição de cada barra
        height = p.get_height()
        # Pegar a cor da barra (cor de preenchimento)
        bar_color = p.get_facecolor()
        # Colocar o valor na ponta da barra
        if height >= 0:
            ax.text(p.get_x() + p.get_width() / 2., height,  # Posição para barras positivas
                    '{:,.0f}'.format(height).replace(',', '.'),
                    ha="center", va="bottom", fontsize=10,
                    fontweight='bold', color=bar_color)
        else:
            ax.text(p.get_x() + p.get_width() / 2., height,  # Posição para barras negativas
                    '{:,.0f}'.format(height).replace(',', '.'),
                    ha="center", va="top", fontsize=10,
                    fontweight='bold', color=bar_color)
            
    plt.figtext(0.05, -0.01, 'Fonte: Novo Caged (MTE).',
                    ha='left', fontsize=8, color='gray', weight='normal')
    plt.figtext(0.05, -0.03, 'Elaboração: SEBRAE/RN.',
                    ha='left', fontsize=8, color='gray', weight='normal')

    # Exibir o gráfico
    plt.tight_layout()

    # Salvar gráfico como arquivo PNG
    grafico_path = f'img/grafico_saldo_porte_{setor}.png'
    plt.savefig(grafico_path, format='png', bbox_inches='tight', transparent=True)

    return grafico_path

grafico_saldo_porte = criar_grafico_saldo_porte(data)

C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_27008\1841820680.py:14: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(x='porte', y='saldomovimentação', data=grouped_data, palette='Blues_d')


In [46]:
caged_nordeste['Estado'].unique()

array(['Maranhão', 'Piauí', 'Ceará', 'Rio Grande do Norte', 'Paraíba',
       'Pernambuco', 'Alagoas', 'Sergipe', 'Bahia'], dtype=object)

In [47]:
import matplotlib.pyplot as plt
import seaborn as sns

def criar_grafico_nordeste(data, setor):
    df_setor = data[data['grande_grupamento'] == setor]
    grouped_data = df_setor.sort_values(by='Saldo', ascending=False)

    cor_destaque = '#3a4e5c'
    cor_padrao = '#6c9dbf'
    estado_destaque = 'Rio Grande do Norte'

    # O seu palette continua perfeito
    estados_ordenados = grouped_data['Estado'].unique()
    palette_cores = {estado: cor_destaque if estado == estado_destaque else cor_padrao for estado in estados_ordenados}

    # --- INÍCIO DA MUDANÇA ---
    
    # 1. Crie uma lista explícita com a ordem dos estados
    #    Essa ordem vem do seu DataFrame já ordenado por 'Saldo'
    ordem_grafico = grouped_data['Estado'].tolist()

    plt.figure(figsize=(12, 6))
    
    ax = sns.barplot(
        x='Estado', 
        y='Saldo', 
        data=grouped_data,
        palette=palette_cores,
        errorbar=None,
        order=ordem_grafico # 2. Adicione este parâmetro!
    )

    # --- FIM DA MUDANÇA ---
    
    plt.axhline(0, color='black', linewidth=1)
    plt.xlabel('')
    plt.ylabel('')

    # AGORA ESTE LOOP FUNCIONA COM SEGURANÇA
    # Porque a ordem do gráfico é a mesma do DataFrame 'grouped_data'
    for bar, estado in zip(ax.patches, ordem_grafico):
        height = bar.get_height()
        color = palette_cores[estado]
        va = 'bottom' if height >= 0 else 'top'

        ax.text(
            bar.get_x() + bar.get_width() / 2.,
            height,
            '{:,.0f}'.format(height).replace(',', '.'),
            ha='center', va=va, fontsize=10,
            fontweight='bold', color=color
        )

    plt.figtext(0.05, -0.01, 'Fonte: Novo Caged (MTE).', ha='left', fontsize=8, color='gray')
    plt.figtext(0.05, -0.03, 'Elaboração: SEBRAE/RN.', ha='left', fontsize=8, color='gray')
    plt.tight_layout()

    grafico_path = f'img/grafico_nordeste_{setor}.png'
    plt.savefig(grafico_path, format='png', bbox_inches='tight', transparent=True)
    plt.close()

    return grafico_path

grafico_nordeste = criar_grafico_nordeste(caged_nordeste, setor)
print(f"Gráfico salvo em: {grafico_nordeste}")


C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_27008\3265797723.py:24: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(


Gráfico salvo em: img/grafico_nordeste_Agropecuária.png


In [48]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import pandas as pd

def criar_grafico_ts_porte(data):
    # Converter a coluna 'competênciamov' para string, caso não seja
    data['competênciamov'] = data['competênciamov'].astype(str)

    # Filtrar os dados onde o ano da 'competênciamov' é igual ao ano de 'competencia'
    ano_competencia = str(competencia)[:4]
    data_ano = data[data['competênciamov'].str[:4] == ano_competencia]

    # Garantir que a coluna 'data' seja do tipo datetime
    data_ano['data'] = pd.to_datetime(data_ano['data'])

    # Agrupar os dados por 'data' e 'porte', somando os valores de 'saldomovimentação'
    grouped_data = data_ano.groupby(['data', 'porte'])['saldomovimentação'].sum().reset_index()

    # Obter a lista de valores únicos de 'porte' e associar cores
    unique_porte = grouped_data['porte'].unique()
    palette = sns.color_palette("muted", n_colors=len(unique_porte))
    color_mapping = dict(zip(unique_porte, palette))

    # Criar o gráfico com hue para a coluna 'porte'
    plt.figure(figsize=(12, 6))
    ax = sns.lineplot(x='data', y='saldomovimentação', hue='porte', data=grouped_data, 
                      marker="o", palette=color_mapping)

    # Títulos e rótulos (ajustáveis conforme necessidade)
    plt.title("")
    plt.xlabel("")
    plt.ylabel("")

    # Adicionar a linha cinza no eixo Y = 0
    plt.axhline(0, color='lightgray', linestyle='--', linewidth=1)
    
    # Formatar os rótulos do eixo X para exibir apenas mês/ano
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  # Exibe de 1 em 1 mês
    plt.xticks(rotation=45)

    # Configurar os ticks do eixo X para garantir que todos os pontos sejam exibidos
    unique_dates = grouped_data['data'].unique()
    plt.xticks(ticks=unique_dates, labels=[date.strftime('%m/%Y') for date in unique_dates])

    # Adicionar valores exatamente no centro dos pontos, com background colorido e números brancos
    for porte in grouped_data['porte'].unique():
        porte_data = grouped_data[grouped_data['porte'] == porte]
        for x, y in zip(porte_data['data'], porte_data['saldomovimentação']):
            # Adicionar texto com um fundo colorido (mesma cor da linha) e texto branco
            plt.text(
                x, y, f'{int(y):,}'.replace(',', '.'),  # Formatação do número
                ha='center', va='center', fontsize=8, color='white',  # Cor do número
                bbox=dict(facecolor=color_mapping[porte], edgecolor='none', boxstyle='round,pad=0.2')  # Fundo colorido
            )

    # Legenda dentro do gráfico no canto superior esquerdo
    plt.legend(
        title="Porte", loc="upper left", frameon=True, fontsize=8,
        title_fontsize=10, facecolor='white', edgecolor='black', fancybox=True
    )

    # Notas no rodapé
    plt.figtext(0.08, -0.01, 'Fonte: Novo Caged (MTE).',
                ha='left', fontsize=8, color='gray', weight='normal')
    plt.figtext(0.08, -0.03, 'Elaboração: SEBRAE/RN.',
                ha='left', fontsize=8, color='gray', weight='normal')

    # Salvar gráfico
    grafico_path = f'img/grafico_ts_porte_{setor}.png'
    plt.savefig(grafico_path, format='png', bbox_inches='tight', transparent=True)
    plt.show()
    return grafico_path

# Uso da função
grafico_ts_porte = criar_grafico_ts_porte(data)


C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_27008\295884125.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ano['data'] = pd.to_datetime(data_ano['data'])


C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_27008\295884125.py:73: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Mapa

In [49]:
data['data'] = pd.to_datetime(data['data'], errors='coerce')

caged_mes_atual = data[data['data'] == data['data'].max()]

municipios = (caged_mes_atual.groupby("município", as_index=False)
    .agg(Saldo=("saldomovimentação", "sum")))

# Add ranking columns
municipios["top5"] = municipios["Saldo"].rank(method="min", ascending=False).isin(range(1, 6))
municipios["bottom5"] = municipios["Saldo"].rank(method="min").isin(range(1, 6))
municipios = municipios.rename(columns={"município": "code_muni"})

# Load municipality map
rn_map = geobr.read_municipality(code_muni="RN")
rn_map['code_muni'] = rn_map['code_muni'].astype(str)
rn_map["code_muni"] = rn_map["code_muni"].str[:-3]

# Ensure both are strings
rn_map['code_muni'] = rn_map['code_muni'].astype(str)
municipios['code_muni'] = municipios['code_muni'].astype(str)

# Now merge
map_saldo = pd.merge(rn_map, municipios, on='code_muni', how = 'outer')

# Formatar os números da coluna 'valores' como inteiros no formato brasileiro
map_saldo['Saldo'] = map_saldo['Saldo'].fillna(0).apply(lambda x: f"{int(x):,}".replace(",", "."))

# Create MUN_SALDO column
map_saldo["MUN_SALDO"] = map_saldo.apply(lambda x: f"{x['name_muni']}\n{x['Saldo']}" if x["top5"] or x["bottom5"] else None, axis=1)

# Defining fill and border colors
fill_palette = ["#3CBC3C", "#C64646", "#d3d3d3"]
border_palette = ["#005800", "#C30000", "#939393"]

map_saldo['fill_colors'] = map_saldo.apply(
    lambda x: fill_palette[0] if x['top5'] else (fill_palette[1] if x['bottom5'] else fill_palette[2]), axis=1
)

map_saldo['border_colors'] = map_saldo.apply(
    lambda x: border_palette[0] if x['top5'] else (border_palette[1] if x['bottom5'] else border_palette[2]), axis=1
)

map_saldo_sf = gpd.GeoDataFrame(map_saldo)

# Filtering top5 and bottom5
top5 = map_saldo_sf[map_saldo_sf['top5'].fillna(False)]
bottom5 = map_saldo_sf[map_saldo_sf['bottom5'].fillna(False)]

# Filtering spatially using bounding boxes
def filter_sf(data, xmin=None, xmax=None, ymin=None, ymax=None):
    bbox = box(xmin, ymin, xmax, ymax) if all([xmin, xmax, ymin, ymax]) else None
    if bbox:
        return data[data.intersects(bbox)]
    return data

# Filtering data to the right of xmin
direita = filter_sf(map_saldo_sf, xmin=-36.6)
esquerda = map_saldo_sf[~map_saldo_sf['code_muni'].isin(direita['code_muni'])]

# Adding text colors
direita['text_colors'] = direita.apply(
    lambda x: "#07ca8d" if x['top5'] else ("#C30000" if x['bottom5'] else None), axis=1
)
esquerda['text_colors'] = esquerda.apply(
    lambda x: "#07ca8d" if x['top5'] else ("#C30000" if x['bottom5'] else None), axis=1
)

# Reproject the GeoDataFrames to a projected CRS (e.g., EPSG:3857)
map_saldo_sf = map_saldo_sf.to_crs(epsg=3857)
top5 = top5.to_crs(epsg=3857)
bottom5 = bottom5.to_crs(epsg=3857)


# Calculate the center longitude of the map
x_center = map_saldo_sf.geometry.centroid.x.mean()

# Add centroid y-coordinates for sorting
top5["centroid_y"] = top5.geometry.centroid.y
bottom5["centroid_y"] = bottom5.geometry.centroid.y

# Split top5 and bottom5 into left (esquerda) and right (direita) based on longitude
top5_direita = top5[top5.geometry.centroid.x > x_center].sort_values(by="centroid_y", ascending=False)
top5_esquerda = top5[top5.geometry.centroid.x <= x_center].sort_values(by="centroid_y", ascending=False)
bottom5_direita = bottom5[bottom5.geometry.centroid.x > x_center].sort_values(by="centroid_y", ascending=False)
bottom5_esquerda = bottom5[bottom5.geometry.centroid.x <= x_center].sort_values(by="centroid_y", ascending=False)

# Colors for top5 and bottom5
fill_colors = {"bottom5": "#C64646", "top5": "#3CBC3C"}
border_colors = {"top5": "#005800", "bottom5": "#C30000"}
darker_colors = {"top5": "#004000", "bottom5": "#8B0000"}  # Darker shades for lines

# Track used positions (lines and boxes) to avoid overlaps
used_positions_right = []
used_positions_left = []

def find_non_overlapping_position(base_y, used_positions, step=16000):
    """
    Find a non-overlapping position for text placement.
    """
    y = base_y
    for _ in range(1000):  # Max attempts
        if all(abs(y - used_y) > step for used_y in used_positions):
            used_positions.append(y)
            return y
        y -= step  # Move downward if necessary to avoid overlaps
    return y  # Fallback

# Plot the map
fig, ax = plt.subplots(figsize=(12, 6))
map_saldo_sf.plot(ax=ax, color="#d3d3d3", edgecolor="#939393", label="Other Municipalities")

# Highlight top5 and bottom5 municipalities
top5.plot(ax=ax, color=fill_colors["top5"], edgecolor=border_colors["top5"], label="Top 5")
bottom5.plot(ax=ax, color=fill_colors["bottom5"], edgecolor=border_colors["bottom5"], label="Bottom 5")

# Function to annotate cities
def annotate_cities(df, side, line_color, box_color, x_offset):
    """
    Annotate cities with labels and connecting lines.
    """
    # Subir mais os textos iniciais ajustando base_y
    base_y = df["centroid_y"].max() + 20000  # Ajuste para mover os números mais para cima
    used_positions = used_positions_right if side == "right" else used_positions_left

    for _, row in df.iterrows():
        centroid = row.geometry.centroid
        y = find_non_overlapping_position(base_y, used_positions, step=14000)  # Menor step para compactar mais os textos
        base_y = y  # Move downward para o próximo rótulo

        # Ajustar a posição do texto com base no lado (direita ou esquerda)
        text_x = x_center + x_offset if side == "right" else x_center - x_offset

        # Desenhar uma linha conectando o centróide ao texto
        ax.plot([centroid.x, text_x], [centroid.y, y], "-", color=line_color, lw=0.8)

        # Adicionar o texto ao lado do mapa
        ax.text(
            text_x, y, row["MUN_SALDO"],
            fontsize=8, ha="left" if side == "right" else "right",
            weight="bold",
            bbox=dict(facecolor="white", edgecolor=box_color, boxstyle="round,pad=0.3")
        )

# Annotate cities on the right (direita)
annotate_cities(top5_direita, side="right", line_color=darker_colors["top5"], box_color=border_colors["top5"], x_offset=200000)
annotate_cities(bottom5_direita, side="right", line_color=darker_colors["bottom5"], box_color=border_colors["bottom5"], x_offset=200000)

# Annotate cities on the left (esquerda)
annotate_cities(top5_esquerda, side="left", line_color=darker_colors["top5"], box_color=border_colors["top5"], x_offset=225000)
annotate_cities(bottom5_esquerda, side="left", line_color=darker_colors["bottom5"], box_color=border_colors["bottom5"], x_offset=225000)

data_maxima = data['competênciamov'].max()

# Converter o número inteiro (YYYYMM) em um objeto datetime
data_maxima_dt = datetime.strptime(str(data_maxima), "%Y%m")

# Extrair ano e mês no formato brasileiro
ano_max = data_maxima_dt.year
mes_max = data_maxima_dt.strftime('%B')  # Traduzir o mês usando o dicionário

# Configurar o título dinamicamente
# ax.set_title(f"Saldo do Emprego em {mes_max} de {ano_max}", fontsize=14)
ax.axis("off")  # Turn off the axis for a cleaner map

# Show the plot
plt.tight_layout()

# Adicionar informações de fonte e elaboração
plt.figtext(0.1, -0.01, 'Fonte: Novo Caged (MTE).',
            ha='left', fontsize=8, color='gray', weight='normal')
plt.figtext(0.1, -0.03, 'Elaboração: SEBRAE/RN.',
            ha='left', fontsize=8, color='gray', weight='normal')

plt.savefig(f"img/map_saldo_emprego_{setor}.png", format="png", dpi=300, bbox_inches="tight", transparent=True)
plt.show()

C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_27008\3522527326.py:46: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  top5 = map_saldo_sf[map_saldo_sf['top5'].fillna(False)]
C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_27008\3522527326.py:47: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bottom5 = map_saldo_sf[map_saldo_sf['bottom5'].fillna(False)]


C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_27008\3522527326.py:175: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


# PDF

In [50]:
# Função para converter a imagem em Base64 (para embutir no HTML)
def imagem_para_base64(caminho_imagem):
    with open(caminho_imagem, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

# Função para gerar o conteúdo HTML
def gerar_html(admissoes, saldo, desligamentos, estoque_clt, setor):
    grafico_acumulado_ano_e_mes =f'img/grafico_acumulado_ano_e_mes_{setor}.png'
    grafico_saldo_ts = f'img/grafico_saldo_ts_{setor}.png'
    grafico_saldo_cnae = f'img/grafico_top_bottom_cnaes_{setor}.png'
    grafico_mes_colunas = f'img/grafico_mes_colunas_{setor}.png'
    grafico_saldo_porte = f'img/grafico_saldo_porte_{setor}.png'
    grafico_nordeste = f'img/grafico_nordeste_{setor}.png'
    grafico_ts_porte = f'img/grafico_ts_porte_{setor}.png'
    mapa = f'img/map_saldo_emprego_{setor}.png'

    # Converter a imagem de cabeçalho em base64
    header_image_base64 = imagem_para_base64('img\header-emprego.png')

    # Converter o gráfico em base64
    grafico_saldo_ts_b64 = imagem_para_base64(grafico_saldo_ts)
    grafico_saldo_cnae_b64 = imagem_para_base64(grafico_saldo_cnae)
    grafico_mes_colunas_b64 = imagem_para_base64(grafico_mes_colunas)
    grafico_saldo_porte_b64 = imagem_para_base64(grafico_saldo_porte)
    grafico_nordeste_b64 = imagem_para_base64(grafico_nordeste)
    grafico_ts_porte_b64 = imagem_para_base64(grafico_ts_porte)
    mapa_b64 = imagem_para_base64(mapa)
    grafico_acumulado_ano_e_mes_b64 = imagem_para_base64(grafico_acumulado_ano_e_mes)

    # Criar conteúdo HTML
    html_content = f"""
    <!DOCTYPE html>
    <html lang="pt-br">
    <head>
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
        <title>Boletim de Emprego</title>
        <style>
            @page {{
                size: 1080px 720px;
                margin: 0;
            }}

            /* Definir o fundo da página */
            body {{
                background-color: #f3f3f3; /* Cor de fundo */
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 0;
            }}
            header {{
                background-image: url('data:image/png;base64,{header_image_base64}');
                background-repeat: no-repeat;
                background-size: 100% 100%;
                background-position: center;
                height: 360px;
                text-align: center;
                color: white;
                font-size: 24px;
                display: flex;
                align-items: center;
                justify-content: center;
            }}
                .mes {{
                font-size: 24px;
                font-weight: bold;
                color: #333;
                text-align: center;
                margin-top: 30px;
            }}

            /* Estilo para as caixinhas */
            .informacoes {{
                display: flex;
                justify-content: space-between;
                padding: 20px;
                margin: 20px 30px;
            }}
            .caixinha {{
                background-color: #fff;
                border: 1px solid #ddd;
                padding: 30px;
                text-align: center;
                width: 22%;
                border-radius: 10px;
                box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            }}
            .caixinha h3 {{
                font-size: 20px;
                color: #1574bc;
            }}
            .caixinha p {{
                font-size: 24px;
                font-weight: bold;
                color: #333;
            }}
            .grafico h3 {{
                font-size: 24px;
                font-weight: bold;
                color: #333;
                text-align: center;
                padding-top: 60px;
            }}

            .grafico h2 {{
                padding-top= -15px;
                font-size: 18px;
                color: #333;
                text-align: center;
                font-weight: normal;
            }}

            .grafico img {{
                width: 1000px;
                padding-top: 20px;
                padding-left: 30px;
            }}

            .tabela {{
                width: 100%;
                border-collapse: collapse;
                margin-top: 20px;
            }}
            th, td {{
                padding: 10px;
                text-align: center;
                border: 1px solid #ddd;
            }}
            th {{
                background-color: #333;
                color: white;
            }}
            td {{
                background-color: #fff;
            }}
            footer {{
                text-align: center;
                font-size: 12px;
                margin-top: 30px;
            }}
        </style>
    </head>
    <body>
        <header>
        </header>

        <!-- Seção do título do mês -->
        <div class="mes">{mes_referente} - {setor}</div>

        <!-- Seção das Caixinhas -->
        <div class="informacoes">
            <div class="caixinha">
                <h3>Admissões</h3>
                <p>{total_admissoes}</p>
            </div>
            <div class="caixinha">
                <h3>Saldo</h3>
                <p>{saldo}</p>
            </div>
            <div class="caixinha">
                <h3>Desligamentos</h3>
                <p>{total_desligamentos}</p>
            </div>
            <div class="caixinha">
                <h3>Estoque CLT</h3>
                <p>{estoque_clt}</p>
            </div>
        </div>
        <div class="grafico">
            <h3>Saldo de Emprego Acumulado e {mes_nome} - {setor}</h3>
            <img src="data:image/png;base64,{grafico_acumulado_ano_e_mes_b64}">
        </div>

        <div class="grafico">
            <h3>Saldo de Emprego por Mês - {setor}</h3>
            <img src="data:image/png;base64,{grafico_saldo_ts_b64}">
        </div>

        <div class="grafico">
            <h3>Top Maiores e Menores Saldos por Atividade Econômica em {ano} - {setor}</h3>
            <img src="data:image/png;base64,{grafico_saldo_cnae_b64}">
        </div>

        <div class="grafico">
            <h3>Saldo de Emprego nos Meses de {mes_nome} - {setor}</h3>
            <img src="data:image/png;base64,{grafico_mes_colunas_b64}">
        </div>

       <div class="grafico">
            <h3>Saldo Acumulado de Emprego de {ano} por Porte - {setor}</h3>
            <img src="data:image/png;base64,{grafico_saldo_porte_b64}">
        </div>

               <div class="grafico">
            <h3>Saldo de Emprego por Estado do Nordeste - {setor} em {mes_referente}</h3>
            <img src="data:image/png;base64,{grafico_nordeste_b64}">
        </div>

        <div class="grafico">
            <h3>Saldo de Emprego por Mês e Porte - {setor}</h3>
            <img src="data:image/png;base64,{grafico_ts_porte_b64}">
        </div>

           <div class="grafico">
            <h3>Saldo de Emprego nos Municípios do RN - {setor} em {mes_referente}</h3>
            <img src="data:image/png;base64,{mapa_b64}">
        </div>
    </body>
    </html>
    """

    return html_content

def gerar_pdf(html_content, file_path):
  weasyprint.HTML(string=html_content).write_pdf(file_path)

# Gerar HTML com os valores das variáveis
html_content = gerar_html(total_admissoes, saldo, total_desligamentos, estoque,setor)

# Gerar o PDF
gerar_pdf(html_content, f"pdfs/boletim_emprego_{competencia}-{setor}.pdf")

<>:18: SyntaxWarning: invalid escape sequence '\h'
<>:18: SyntaxWarning: invalid escape sequence '\h'
C:\Users\lhayane.macedo\AppData\Local\Temp\ipykernel_27008\3353511255.py:18: SyntaxWarning: invalid escape sequence '\h'
  header_image_base64 = imagem_para_base64('img\header-emprego.png')
